## Todo
- chunk them out by 30 seconds to bootstrap and create more samples
- try some cross validation 
- test for overfitting
- more features 
    - get time signature from meta messages
    - ~~stdev of velocity (instead of just average)~~
    - create some manual cross variables with timing and key and time sig
    - frequency domain features

## Initial Imports and Paths

In [1]:
from composer_class_funcs import *

In [2]:
# ml packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [3]:
# file paths
train_midi_path = "./Challenge_DataSet/PS1/"
test_midi_path = "./Challenge_DataSet/PS2/"

## Data Collection and Processing

In [4]:
file_path_ps2 = test_midi_path+"0.981087291054314_adj.mid"
file_path_ps1 = train_midi_path+"Bach/Cello Suite 3_BWV1009_2217_cs3-1pre.mid"
x = extract_features_from_midi(file_path_ps2)

In [43]:
# import file
midi = mido.MidiFile(file_path_ps1)

# initialize values
note_counts = [0] * 128  # MIDI notes range from 0 to 127
# total_velocity = 0
velocities = []
note_on_count = 0
elapsed_time = 0
key = '' # each file should have only 1 key. Investigate if this assumption is correct.
tpb = midi.ticks_per_beat
type = midi.type
filename = os.path.basename(midi.filename)

In [44]:
second_interval = [60,90]
print(second_interval[0])
print(second_interval[1])

60
90


In [45]:
# get ticks
for msg in midi:
    # get the key
    if msg.is_meta and msg.type == 'key_signature':
        key = msg.key
    
    # just the first n seconds
    elapsed_time += msg.time
    if (elapsed_time>=second_interval[0] and elapsed_time<second_interval[1]):
        if msg.type == 'note_on' and msg.velocity > 0:
            note_counts[msg.note] += 1
            # total_velocity += msg.velocity
            velocities.append(msg.velocity)
            note_on_count += 1
    # else:
    #     break

In [46]:

        
# Calculate velocity statistics
if velocities:
    average_velocity = np.mean(velocities)
    variance_velocity = np.var(velocities)
else:
    average_velocity = 0
    variance_velocity = 0

# Normalize the note counts to be between 0 and 1
normalized_note_counts = (note_counts - np.min(note_counts)) / (np.max(note_counts) - np.min(note_counts))

# combine into 1 list
combined_features = [filename, type, tpb, key, average_velocity, variance_velocity] + list(normalized_note_counts)

In [ ]:
features, labels = load_dataset(train_midi_path, labeled=True)
df_labeled = create_dataframe(features, labels)

In [ ]:
unlabeled_features = load_dataset(test_midi_path, labeled=False)
df_unlabeled=create_dataframe(unlabeled_features)

In [ ]:
numeric_columns = df_labeled.select_dtypes(include=['float64', 'int64']).columns

## EDA

In [ ]:
df_labeled.head()

In [ ]:
df_unlabeled.head()

In [ ]:
# Display basic information about the DataFrame
print("\nBasic Information about the DataFrame:")
print(df_labeled.info())

# Generate summary statistics
print("\nSummary Statistics of the DataFrame:")
print(df_labeled.describe())

# Check for missing values
print("\nMissing Values in the DataFrame:")
print(df_labeled.isnull().sum())

In [ ]:
# confirm they're all type 1: 
## https://mido.readthedocs.io/en/latest/files/midi.html#file-types
## type 1 (synchronous): all tracks start at the same time
print(df_labeled.type.value_counts())
print(df_unlabeled.type.value_counts())

In [ ]:
df_labeled.key.value_counts()

In [ ]:
# Visualize the distribution of each numeric feature
# numeric_columns = df_labeled.select_dtypes(include=['float64', 'int64']).columns

plt.figure(figsize=(20, 15))
for i, col in enumerate(numeric_columns):
    plt.subplot(10, 14, i+1)
    sns.histplot(df_labeled[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the distribution of each numeric feature
# numeric_columns = df_unlabeled.select_dtypes(include=['float64', 'int64']).columns

plt.figure(figsize=(20, 15))
for i, col in enumerate(numeric_columns):
    plt.subplot(10, 14, i+1)
    sns.histplot(df_unlabeled[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

based on this, I'd remove notes 0-22, 105-127.
The unlabeled dataset also has no info for these notes.
See feature engineering and prep section for removal

In [ ]:
plt.figure(figsize=(20, 15))
for i, col in enumerate(numeric_columns):
    plt.subplot(10, 14, i+1)
    sns.histplot(df_labeled[col], kde=True)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize correlations between numeric features
plt.figure(figsize=(12, 10))
correlation_matrix = df_labeled[numeric_columns].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
numeric_columns

In [ ]:
# Visualize the relationship between the first two numeric features and the target (if applicable)
if 'composer' in df_labeled.columns:
    for i in range(1,4):
        plt.figure(figsize=(10, 6))
        sns.boxplot(x='composer', y=numeric_columns[i], data=df_labeled, hue='composer')
        plt.title(f'{numeric_columns[i]} by Composer')
        plt.show()

In [ ]:
# Prepare the data for the stacked bar chart
key_composer_counts = df_labeled.groupby(['key', 'composer']).size().unstack(fill_value=0)
# Plot the stacked bar chart
key_composer_counts.plot(kind='bar', stacked=True, figsize=(14, 7), colormap='viridis')
plt.title('Number of Songs per Key, Colored by Composer')
plt.xlabel('Key')
plt.ylabel('Number of Songs')
plt.xticks(rotation=90)
plt.legend(title='Composer')
plt.show()

## Clean + Feature engineer

In [ ]:
# Drop columns corresponding to notes 0-22 and 105-127
cols_to_drop = [f'Note_{i}' for i in list(range(0, 23)) + list(range(105, 128))]
df_labeled.drop(columns=cols_to_drop, inplace=True)
df_unlabeled.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# replace Null keys with 'unk' value
df_labeled['key']=df_labeled['key'].fillna('unk')
df_unlabeled['key']=df_unlabeled['key'].fillna('unk')

# Encode the 'key' variable
label_encoder_key = LabelEncoder()
df_labeled['key_encoded'] = label_encoder_key.fit_transform(df_labeled['key'])
df_unlabeled['key_encoded'] = label_encoder_key.transform(df_unlabeled['key']) # only transform to use the same encoding as labeled

# Encode the 'Composer' column
label_encoder_composer = LabelEncoder()
df_labeled['composer'] = label_encoder_composer.fit_transform(df_labeled['composer'])

In [ ]:
# Define the features (X) and target (y)
X = df_labeled.drop(columns=['composer', 'filename','key'])
y = df_labeled['composer']

# Define features for unlabeled data
z = df_unlabeled.drop(columns=['filename','key'])

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
print(label_encoder_composer.classes_,'\n')
print('train targets\n',y_train.value_counts())
print('\ntest targets\n',y_test.value_counts())

## Train Classifiers

### Logistic Regression

In [ ]:
# Train and evaluate the Logistic Regression model
log_reg = LogisticRegression(max_iter=10000, random_state=42)
log_reg.fit(X_train, y_train)
# predictions
y_pred_test_lr = log_reg.predict(X_test)
y_pred_train_lr = log_reg.predict(X_train)
# Get the classification probabilities for each class for test dataset
y_proba_test_lr = log_reg.predict_proba(X_test)
y_proba_train_lr = log_reg.predict_proba(X_train)

In [ ]:
model_eval("Logistic Regression", y_train, y_pred_train_lr, y_proba_train_lr, y_test, y_pred_test_lr, y_proba_test_lr, label_encoder_composer)

In [ ]:
# model_eval("Logistic Regression", y_test, y_pred_test_lr, y_proba_lr, label_encoder_composer)

### SVM

In [ ]:
# Build and train the Random Forest classifier
svm_classifier = LinearSVC(penalty='l2', random_state=0, tol=1e-5)
# Wrap the LinearSVC classifier with CalibratedClassifierCV to obtain probabilities
calibrated_svc = CalibratedClassifierCV(estimator=svm_classifier, method='sigmoid')
calibrated_svc.fit(X_train, y_train)
# predictions
y_pred_test_svc = calibrated_svc.predict(X_test)
y_pred_train_svc = calibrated_svc.predict(X_train)
# Get the classification probabilities for each class for test dataset
y_proba_test_svc = calibrated_svc.predict_proba(X_test)
y_proba_train_svc = calibrated_svc.predict_proba(X_train)

In [ ]:
# model_eval("Support Vector Machine (CV Calibrated)", y_test, y_pred_svc, y_proba_svc, label_encoder_composer)
model_eval("Support Vector Machine (CV Calibrated)", y_train, y_pred_train_svc, y_proba_train_svc, y_test, y_pred_test_svc, y_proba_test_svc, label_encoder_composer)

### Random Forest

In [ ]:
# Build and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
# predictions
y_pred_test_rf = rf_classifier.predict(X_test)
y_pred_train_rf = rf_classifier.predict(X_train)
# Get the classification probabilities for each class for test dataset
y_proba_test_rf = rf_classifier.predict_proba(X_test)
y_proba_train_rf = rf_classifier.predict_proba(X_train)

In [ ]:
# model_eval("Random Forest", y_test, y_pred_rf, y_proba_rf, label_encoder_composer)
model_eval("Random Forest", y_train, y_pred_train_rf, y_proba_train_rf, y_test, y_pred_test_rf, y_proba_test_rf, label_encoder_composer)

### GBM

In [ ]:
# Build and train the Random Forest classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_classifier.fit(X_train, y_train)

# predictions
y_pred_test_gb = gb_classifier.predict(X_test)
y_pred_train_gb = gb_classifier.predict(X_train)
# Get the classification probabilities for each class for test dataset
y_proba_test_gb = gb_classifier.predict_proba(X_test)
y_proba_train_gb = gb_classifier.predict_proba(X_train)

# # Predict the target on the test set
# y_pred_gb = gb_classifier.predict(X_test)
# # Get the classification probabilities for each class
# y_proba_gb = gb_classifier.predict_proba(X_test)

In [ ]:
# model_eval("GBM", y_test, y_pred_gb, y_proba_gb, label_encoder_composer)
model_eval("GBM", y_train, y_pred_train_gb, y_proba_train_gb, y_test, y_pred_test_gb, y_proba_test_gb, label_encoder_composer)

## Inference: Unlabeled

In [ ]:
# Predict the target on the test set
y_pred_unlabeled = gb_classifier.predict(z)
# Get the classification probabilities for each class
y_proba_unlabeled = gb_classifier.predict_proba(z)

In [ ]:
print(y_pred_unlabeled)
print(label_encoder_composer.inverse_transform(y_pred_unlabeled))
# print(y_proba_unlabeled)

In [ ]:
# Prepare the data for plotting
df_plot = pd.DataFrame(y_proba_unlabeled, columns=[f'Class_{i}_prob' for i in range(y_proba_unlabeled.shape[1])])
df_plot['Predicted_Class'] = y_pred_unlabeled
df_plot['Predicted Composer'] = label_encoder_composer.inverse_transform(y_pred_unlabeled)

# Extract the highest predicted probability for each record
df_plot['Max_Probability'] = df_plot[[f'Class_{i}_prob' for i in range(y_proba_unlabeled.shape[1])]].max(axis=1)

# Plot the histogram
plt.figure(figsize=(12, 8))
sns.histplot(data=df_plot, x='Max_Probability', hue='Predicted Composer', multiple='stack', palette='tab10', bins=20)
plt.title('Histogram of Predicted Probabilities by Class')
plt.xlabel('Predicted Probability')
plt.ylabel('Number of Records')
plt.tight_layout()
plt.show()

Based on this histogram, we can select a probability threshold such that
```
if max(predicted probability for all classes) < threshold
then midi file is NOT one of the 4 composers in training data
````
To begin, I'd select a threshold of 0.90, resulting in 7 out of the 35 midi files being classified as NOT belonging to our 4 known composers.

In [ ]:
df_unlabeled[['filename']].merge(df_plot[df_plot.Max_Probability<0.90],left_index=True, right_index=True, how = 'right').drop(
    labels=['Predicted_Class','Predicted Composer'], axis=1)

### How we would operationalize/functionalize

# scratch

In [ ]:
file_path_ps2 = test_midi_path+"0.981087291054314_adj.mid"
file_path_ps1 = train_midi_path+"Bach/Cello Suite 3_BWV1009_2217_cs3-1pre.mid"

In [ ]:
x = extract_features_from_midi(file_path_ps1)